In [1]:
%run ./Utils.ipynb

In [2]:
# Load dataset function

def load_dataset():
    train_X = np.load(f"train_{COLLECTION_FILE_NAME}_x.npy")
    train_Y = np.load(f"train_{COLLECTION_FILE_NAME}_y.npy")
    test_X = np.load(f"test_{COLLECTION_FILE_NAME}_x.npy")
    test_Y = np.load(f"test_{COLLECTION_FILE_NAME}_y.npy")
    return train_X, train_Y, test_X, test_Y
    
X_train, Y_train, X_test, Y_test = load_dataset() 
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(320, 10304) (320,) (80, 10304) (80,)


In [3]:
# Scale images

X_train = X_train/255.
X_test  = X_test/255.

In [4]:
# Function for encode categorical labels

from sklearn.preprocessing import OneHotEncoder

def encode_labels(df_train, df_test):
    df = np.concatenate((df_train, df_test))
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    df = enc.fit_transform((df).reshape((-1, 1)))
    return df[:-80], df[320:]

Y_train_encoded, Y_test_encoded = encode_labels(Y_train, Y_test)
print(Y_train_encoded.shape, Y_test_encoded.shape)

(320, 40) (80, 40)


In [14]:
# PCA REALIZATION
from sklearn.decomposition import PCA

pca = PCA(.95)
pca.fit(X_train)

X_train_PCA = pca.transform(X_train)
X_test_PCA  = pca.transform(X_test)
print(train_img.shape, test_img.shape)

(320, 161) (80, 161)


In [15]:
# Implementation of NN with helper functions

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def initialize_with_zeros(dim):
    w = np.zeros((dim, 40))
    b = 0

    assert(w.shape == (dim, 40))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)                           # compute activation
#     cost = - np.mean(Y * np.log(A) + (1 - Y) * np.log(1 - A)) # compute cost
    cost = np.mean((Y - A)**2)
    dw = np.dot(X, (A - Y).T) / m
    db = np.mean(A - Y)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)

        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)

        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs


def predict(w, b, X):
    m = X.shape[1]
    
    w = w.reshape(X.shape[0], 40)
    
    A = sigmoid(np.dot(w.T, X) + b)
    Y_prediction = np.round(A)
        
    assert(Y_prediction.shape == (40, m))
    
    return Y_prediction

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    w, b = initialize_with_zeros(X_train.shape[0])

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [17]:
d = model(X_train_PCA.T, Y_train_encoded.T, X_test_PCA.T, Y_test_encoded.T, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.250000
Cost after iteration 100: 0.192958
Cost after iteration 200: 0.152551
Cost after iteration 300: 0.122233
Cost after iteration 400: 0.099266
Cost after iteration 500: 0.081693
Cost after iteration 600: 0.068094
Cost after iteration 700: 0.057441
Cost after iteration 800: 0.048994
Cost after iteration 900: 0.042215
Cost after iteration 1000: 0.036713
Cost after iteration 1100: 0.032197
Cost after iteration 1200: 0.028454
Cost after iteration 1300: 0.025321
Cost after iteration 1400: 0.022676
Cost after iteration 1500: 0.020425
Cost after iteration 1600: 0.018495
Cost after iteration 1700: 0.016827
Cost after iteration 1800: 0.015378
Cost after iteration 1900: 0.014111
train accuracy: 99.9765625 %
test accuracy: 99.46875 %
